In [6]:
import os

from dotenv import load_dotenv
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load environment variables from .env
load_dotenv()

# Define the persistent directory
current_dir = os.path.dirname(os.path.abspath('Learn-LangChain'))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")

# Define the embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# Create a model
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Function to simulate a continual chat
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    chat_history = []  # Collect chat history here (a sequence of messages)
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break
        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # Display the AI's response
        print(f"AI: {result['answer']}")
        # Update the chat history
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))

In [13]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F3E8974E10>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT ans

1. Xác định ngữ cảnh cho câu hỏi (Contextualizing Questions)
Phần này tập trung vào cách hệ thống xử lý các câu hỏi tiếp theo trong một cuộc trò chuyện.

- contextualize_q_system_prompt: Đây là một lời nhắc hệ thống (system prompt) hướng dẫn Mô hình Ngôn ngữ Lớn (LLM) về vai trò của nó. Mục đích của nó là lấy câu hỏi hiện tại của người dùng và lịch sử trò chuyện đang diễn ra, sau đó tạo ra một câu hỏi độc lập có thể hiểu được mà không cần đến lịch sử trò chuyện. Điều này rất quan trọng vì nếu người dùng hỏi một câu hỏi tiếp theo như "Cái đó thì sao?", hệ thống cần hiểu "cái đó" đề cập đến điều gì trong ngữ cảnh của cuộc trò chuyện trước đó. Lời nhắc này đặc biệt yêu cầu LLM không trả lời câu hỏi, chỉ cần diễn đạt lại nó.

- contextualize_q_prompt: Mã này tạo ra một ChatPromptTemplate sử dụng lời nhắc hệ thống. Nó cũng bao gồm MessagesPlaceholder("chat_history") và ("human", "{input}").
MessagesPlaceholder("chat_history"): Điều này cho biết mẫu lời nhắc sẽ chèn lịch sử trò chuyện thực tế (một danh sách các tin nhắn) vào vị trí này khi lời nhắc được tạo.
("human", "{input}"): Điều này đại diện cho câu hỏi mới nhất từ người dùng. Mẫu này sẽ được sử dụng để định dạng đầu vào cho LLM khi nó cố gắng diễn đạt lại một câu hỏi.
- history_aware_retriever: Đây là một thành phần quan trọng.

- create_history_aware_retriever kết hợp:
llm: Mô hình ngôn ngữ lớn của bạn (ví dụ: GPT của OpenAI, Gemini của Google, v.v.).
retriever: Một đối tượng chịu trách nhiệm tìm nạp các tài liệu liên quan từ cơ sở tri thức của bạn dựa trên một truy vấn. (Đối tượng retriever không được định nghĩa trong đoạn mã bạn cung cấp nhưng được giả định là đã tồn tại).
contextualize_q_prompt: Mẫu lời nhắc được định nghĩa ở trên. Thiết lập này có nghĩa là trước khi tìm nạp tài liệu, LLM sẽ sử dụng contextualize_q_prompt và chat_history để diễn đạt lại câu hỏi input của người dùng thành một truy vấn độc lập, rõ ràng. Truy vấn được diễn đạt lại này sau đó được chuyển đến retriever để tìm thông tin liên quan.


- qa_system_prompt: Đây là một lời nhắc hệ thống khác, nhưng lời nhắc này hướng dẫn LLM cách trả lời câu hỏi sau khi ngữ cảnh liên quan đã được tìm nạp. Nó nêu rõ:
Vai trò của LLM là "trợ lý trả lời câu hỏi".
Nó phải sử dụng context được cung cấp (sẽ được chèn vào {context}).
Nếu câu trả lời không có trong ngữ cảnh, nó nên nói "Tôi không biết".
Câu trả lời phải ngắn gọn, "tối đa ba câu".
- qa_prompt: Tương tự như contextualize_q_prompt, điều này tạo ra một ChatPromptTemplate cho giai đoạn trả lời câu hỏi. Nó bao gồm qa_system_prompt, chat_history và input gốc của người dùng. Giá trị giữ chỗ {context} trong qa_system_prompt sẽ được điền bằng các tài liệu được tìm nạp bởi history_aware_retriever.
- Youtube_chain: Điều này sử dụng create_stuff_documents_chain. Thuật ngữ "stuff" (nhồi nhét) đề cập đến một chiến lược phổ biến trong RAG, nơi tất cả các tài liệu đã tìm nạp được "nhồi nhét" (tức là nối và chèn) trực tiếp vào lời nhắc làm ngữ cảnh cho LLM. Chuỗi này lấy llm và qa_prompt để tạo câu trả lời dựa trên các tài liệu được cung cấp.

rag_chain: Đây là chuỗi Retrieval-Augmented Generation chính. create_retrieval_chain kết nối history_aware_retriever với Youtube_chain. Đây là cách nó hoạt động từ đầu đến cuối:
- Một câu hỏi input của người dùng được đưa vào.
- history_aware_retriever (sử dụng contextualize_q_prompt và llm) đầu tiên diễn đạt lại input thành một câu hỏi độc lập, có tính đến chat_history.
- Câu hỏi được diễn đạt lại này sau đó được sử dụng bởi retriever (một phần của history_aware_retriever) để tìm nạp các tài liệu liên quan từ cơ sở tri thức của bạn.
- Các tài liệu được tìm nạp này được chuyển làm context cho Youtube_chain.
- Youtube_chain (sử dụng qa_prompt và llm) sau đó tạo ra một câu trả lời ngắn gọn cho câu hỏi input gốc, chỉ dựa vào context được cung cấp và tuân thủ các ràng buộc đã chỉ định (ví dụ: tối đa ba câu).

In [7]:
continual_chat()

Start chatting with the AI! Type 'exit' to end the conversation.


You:  who is romeo


AI: Romeo is a fictional character and the titular protagonist of William Shakespeare's famous tragedy *Romeo and Juliet*.  He is a Montague, a member of a family locked in a bitter feud with the Capulets.  His love for Juliet, a Capulet, ultimately leads to their tragic deaths.



You:  do you know what i asked


AI: Yes, you asked who Romeo is.



You:  q


AI: That's not a question.  Is there something else I can help you with?



You:  exit


In [11]:
chat_history = [HumanMessage('who is romeo'), AIMessage("Romeo is a fictional character and the titular protagonist of William Shakespeare's famous tragedy *Romeo and Juliet*.  He is a Montague, a member of a family locked in a bitter feud with the Capulets.  His love for Juliet, a Capulet, ultimately leads to their tragic deaths.")]
history_aware_retriever.invoke({'})

AttributeError: 'str' object has no attribute 'get'